# Isotropic vortex

In this notebook, we will model an isotropic vortex as a solution to Euler equations in 2D using Clawpack. The model is example 2.2 from paper ["On the Order of Accuracy and Numerical Performance of Two Classes of Finite Volume WENO Schemes" by Zhang et al](https://www-cambridge-org.colorado.idm.oclc.org/core/journals/communications-in-computational-physics/article/abs/on-the-order-of-accuracy-and-numerical-performance-of-two-classes-of-finite-volume-weno-schemes/30DE290D74891895A66456B940DF2A5E). The code is based on the previous notebook for PyClaw
[Quadrants by David Ketcheson](https://github.com/clawpack/apps/blob/master/notebooks/pyclaw/Quadrants.ipynb).

## Set-up 

The general Euler equation for compressible flow has the following form:
$u_t+f(u)_x+g(u)_y=0$ 
with
$$u=
\begin{bmatrix}
\rho\\ \rho v\\ \rho w\\ E
\end{bmatrix},\quad
f(u)=\begin{bmatrix}
\rho v\\ \rho v^2+P\\ \rho vw\\ v(E+P)
\end{bmatrix},\quad
g(u)=\begin{bmatrix}
\rho w\\ \rho vw\\ \rho w^2+P\\ w(E+P)
\end{bmatrix},
$$

where $\rho$ is the density, $v$, $w$ are the velocities, $E$ is the total energy and $P$ is the pressure. The boundary conditions are periodic. The following code sets up a SharpClaw solver for the Euler equations, on the domain $(x,y)\in [0,10]\times[0,10]$.

In [ ]:
#loading necessary packages
from clawpack import pyclaw
from clawpack import riemann
import matplotlib.pyplot as plt
import numpy as np

clawSC = pyclaw.Controller()
clawSC.tfinal = 10. # at t=10 the vortex will have returned to its original position
clawSC.num_output_times = 100

riemann_solver = riemann.euler_4wave_2D
clawSC.solver = pyclaw.SharpClawSolver2D(riemann_solver)
clawSC.solver.all_bcs = pyclaw.BC.periodic

grid_size = (50,50)
domain = pyclaw.Domain( (0.,0.), (10.,10.), grid_size)

## Vortex evolution

Let $\epsilon$ be the vortex strength, $T=P/\rho$ be the temperature, and $S=P/\rho^\gamma$ be the enthalpy ($\gamma$ is the common gas constant fixed at $1.4$). The center of the vortex will be initially positioned at $(x,y)=(5,5)$. The ambient flow values are $\rho=1,\,P=1,\,v=1,\,w=1$. The vortex is initialized by introducing perturbations to $v,\,w,\,T,\,S$ (denoted by $\delta$):

$$
\begin{bmatrix}
\delta v\\ \delta w\\ \delta T\\ \delta S
\end{bmatrix}=
\begin{bmatrix}
-\frac{\epsilon}{2\pi}e^{0.5(1-r^2)}(y-5)^2\\
\frac{\epsilon}{2\pi}e^{0.5(1-r^2)}(x-5)^2\\
-\frac{(\gamma-1)\epsilon^2}{8\gamma\pi^2}e^{1-r^2}\\
0
\end{bmatrix}.
$$

In Clawpack, the initial conditions need to be specified in terms of the conservated quantities (density, momentum, and energy). To compute these values we made use of [the vortex example in libCEED](https://github.com/CEED/libCEED/blob/main/examples/fluids/qfunctions/eulervortex.h#L92-L107). Using the mean flow values, we get that $T=1+\delta T$ and $S_0=1$.  By the formulae for $T$ and $S$ we get $\rho=(T/S)^{\gamma-1}$. The initial conditions are as follows:
$$
\begin{bmatrix}
\rho\\
\rho(1+\delta v)\\
\rho(1+\delta w)\\
0.5\rho((1+\delta v)^2+(1+\delta w)^2)+\frac{\rho T}{\gamma-1}
\end{bmatrix}.
$$

The code below sets up the initial conditions and SharpClaw (via PyClaw) is used to solve this problem.

In [ ]:
#here we find the solution and set the parameters
clawSC.solution = pyclaw.Solution(clawSC.solver.num_eqn,domain)
gam = 1.4
clawSC.solution.problem_data['gamma']  = gam

# Set initial data
q = clawSC.solution.q
xx,yy = domain.grid.p_centers
eps = 5 #vortex strength
xbar = xx-5.0
ybar = yy-5.0
r2 = xbar**2+ybar**2 # distance from vortex center
A = eps/(2*np.pi)*np.exp(0.5*(1.0-r2)) # velocity perturbation
dT = -(gam-1.)*eps**2/(8*gam*np.pi**2)*np.exp(1-r2) # temperature perturbation
S_vor = 1 # initial enthalpy, no perturbation
T = 1+dT # initial temperature + perturbation
rho = pow(T / S_vor, 1 / (gam - 1.)); # initial density
P = rho*T # initial pressure
c_v = 1 # mean velocity in x
c_w = 1 # mean velocity in y

# Set initial solution
q[0,...] = rho
q[1,...] = rho*(c_v-A*ybar)
q[2,...] = rho*(c_w+A*xbar)
q[3,...] = 0.5*rho*((c_v-A*ybar)**2+(c_w+A*xbar)**2) + P/(gam-1.)

clawSC.keep_copy = True       # Keep solution data in memory for plotting
clawSC.output_format = None   # Don't write solution data to file
clawSC.solver.dt_initial=1.e99
status = clawSC.run()

## Visualizing results

Next following similar tutorials and examples, we will visialize the results. First, the animation of the evolution of the density is presented. As expected it moves diagonally, however as it evolves and crosses the boundary, we observe that surrounding are the vortex are not constant (when they should be). Ideally, values at $t=0$ and $t=10$ should be the same as one period passes. Hence, in the second part, we plot gradient densities for the initial and final frame. 

Theoretically the initial and the final solution should be the same after 1 period, hence how the norm is computed in the end. We focus on density in this notebook, but other quantities would produce similar results.

In [ ]:
from clawpack.visclaw import ianimate
ianimate.ianimate(clawSC)

In [ ]:
%matplotlib inline

frameSC_init = clawSC.frames[0]
densitySC_init = frameSC_init.q[0,:,:]
(vxSC_init,vySC_init) = np.gradient(densitySC_init)
vsSC_init = np.sqrt(vxSC_init**2 + vySC_init**2)
x, y = frameSC_init.state.grid.c_centers 
frameSC_fin = clawSC.frames[-1]
densitySC_fin = frameSC_fin.q[0,:,:]
(vxSC_fin,vySC_fin) = np.gradient(densitySC_fin)
vsSC_fin = np.sqrt(vxSC_fin**2 + vySC_fin**2)  

figSC,(axSC1, axSC2) = plt.subplots(1, 2)
figSC.suptitle('Schlieren plots for SharpClaw')
axSC1.pcolormesh(x, y, vsSC_init, cmap='RdBu')
axSC1.set_aspect('equal')
axSC1.set_title('Initial, t=0')
axSC2.pcolormesh(x, y, vsSC_fin, cmap='RdBu')
axSC2.set_aspect('equal')
axSC2.set_title('Final, t=10');

In [ ]:
dxdy = np.prod(clawSC.solution.state.grid.delta)
diff = np.linalg.norm((densitySC_init-densitySC_fin).flatten(),ord=1)*dxdy
print('Norm of the difference between initial and final density: ', diff)

Now we compare performance with the 2nd-order Lax-Wendroff-based Clawpack implementation.

In [ ]:
clawCP = pyclaw.Controller()
clawCP.tfinal = 10.
clawCP.num_output_times =100

clawCP.solver = pyclaw.ClawSolver2D(riemann_solver)
clawCP.solver.all_bcs = pyclaw.BC.periodic

clawCP.solution = pyclaw.Solution(clawCP.solver.num_eqn,domain)
clawCP.solution.problem_data['gamma']  = gam

# Set initial data
q = clawCP.solution.q
q[0,...] =rho
q[1,...] = rho*(c_v-A*ybar)
q[2,...] = rho*(c_w+A*xbar)
q[3,...] = 0.5*rho*((c_v-A*ybar)**2+(c_w+A*xbar)**2) + P/(gam-1.)

clawCP.keep_copy = True       # Keep solution data in memory for plotting
clawCP.output_format = None   # Don't write solution data to file
clawCP.solver.dt_initial=1.e99
status = clawCP.run()

In [ ]:
ianimate.ianimate(clawCP)

In [ ]:
frameCP_init = clawCP.frames[0]
densityCP_init = frameCP_init.q[0,:,:]
(vxCP_init,vyCP_init) = np.gradient(densityCP_init)
vsCP_init = np.sqrt(vxCP_init**2 + vyCP_init**2)
#x, y = frameCP_init.state.grid.c_centers 
frameCP_fin = clawCP.frames[-1]
densityCP_fin = frameCP_fin.q[0,:,:]
(vxCP_fin,vyCP_fin) = np.gradient(densityCP_fin)
vsCP_fin = np.sqrt(vxCP_fin**2 + vyCP_fin**2)  

figCP,(axCP1, axCP2) = plt.subplots(1, 2)
figCP.suptitle('Schlieren plots for Clawpack')
axCP1.pcolormesh(x, y, vsCP_init, cmap='RdBu')
axCP1.set_aspect('equal')
axCP1.set_title('Initial, t=0')
axCP2.pcolormesh(x, y, vsCP_fin, cmap='RdBu')
axCP2.set_aspect('equal')
axCP2.set_title('Final, t=10');

In [ ]:
diff = np.linalg.norm((vsCP_init-vsCP_fin).flatten(),ord=1)*dxdy
print('Norm of the difference between initial and final density: ', diff)

It is interesting to see the discrepancies between SharpClaw and standard Clawpack results. To investigate it further we perform an error analysis. The error would be a matrix norm of difference between the initial and the final density gradients (can also look at other variables). We will vary the spatial grid size, solve using both methods and report the error.

In [ ]:
errSC=[0]*10
for i in range (1,11):
    clawSC = pyclaw.Controller()
    clawSC.tfinal = 10.
    clawSC.num_output_times =100

    clawSC.solver = pyclaw.SharpClawSolver2D(riemann_solver)
    clawSC.solver.all_bcs = pyclaw.BC.periodic

    grid_size = (i*10,i*10)
    domain = pyclaw.Domain( (0.,0.), (10.,10.), grid_size)

    clawSC.solution = pyclaw.Solution(clawSC.solver.num_eqn,domain)
    gam = 1.4
    clawSC.solution.problem_data['gamma']  = gam

# Set initial data
    q = clawSC.solution.q
    xx,yy = domain.grid.p_centers
    eps=5 #vortex strength
    xbar=xx-5.0
    ybar=yy-5.0
    r2=xbar**2+ybar**2
    A=eps/(2*np.pi)*np.exp(0.5*(1.0-r2))
    dT=-(gam-1.)*eps**2/(8*gam*np.pi**2)*np.exp(1-r2)
    S_vor=1
    T=1+dT
    rho  = pow(T / S_vor, 1 / (gam - 1.));
    P=rho*T
    c_v=1
    c_w=1

    q[0,...] =rho
    q[1,...] = rho*(c_v-A*ybar)
    q[2,...] = rho*(c_w+A*xbar)
    q[3,...] = 0.5*rho*((c_v-A*ybar)**2+(c_w+A*xbar)**2) + P/(gam-1.)

    clawSC.keep_copy = True      # Keep solution data in memory for plotting
    clawSC.output_format = None   # Don't write solution data to file
    clawSC.solver.dt_initial=1.e99
    status = clawSC.run()
    frameSC_init = clawSC.frames[0]
    densitySC_init = frameSC_init.q[0,:,:]
    #(vxSC_init,vySC_init) = np.gradient(densitySC_init)
    #vsSC_init = np.sqrt(vxSC_init**2 + vySC_init**2)
    frameSC_fin = clawSC.frames[-1]
    densitySC_fin = frameSC_fin.q[0,:,:]
    #(vxSC_fin,vySC_fin) = np.gradient(densitySC_fin)
    #vsSC_fin = np.sqrt(vxSC_fin**2 + vySC_fin**2)  
    errSC[i-1]=np.linalg.norm((densitySC_init-densitySC_fin).flatten(),ord=1)*10**2/(10*i)**2

In [ ]:
errCP=[0]*10
for i in range (1,11):
    clawCP = pyclaw.Controller()
    clawCP.tfinal = 10.
    clawCP.num_output_times =100

    clawCP.solver = pyclaw.ClawSolver2D(riemann_solver)
    clawCP.solver.all_bcs = pyclaw.BC.periodic

    grid_size = (i*10,i*10)
    domain = pyclaw.Domain( (0.,0.), (10.,10.), grid_size)

    clawCP.solution = pyclaw.Solution(clawCP.solver.num_eqn,domain)
    gam = 1.4
    clawCP.solution.problem_data['gamma']  = gam

# Set initial data
    q = clawCP.solution.q
    xx,yy = domain.grid.p_centers
    eps=5 #vortex strength
    xbar=xx-5.0
    ybar=yy-5.0
    r2=xbar**2+ybar**2
    A=eps/(2*np.pi)*np.exp(0.5*(1.0-r2))
    dT=-(gam-1.)*eps**2/(8*gam*np.pi**2)*np.exp(1-r2)
    S_vor=1
    T=1+dT
    rho  = pow(T / S_vor, 1 / (gam - 1.));
    P=rho*T
    c_v=1
    c_w=1

    q[0,...] =rho
    q[1,...] = rho*(c_v-A*ybar)
    q[2,...] = rho*(c_w+A*xbar)
    q[3,...] = 0.5*rho*((c_v-A*ybar)**2+(c_w+A*xbar)**2) + P/(gam-1.)

    clawCP.keep_copy = True      # Keep solution data in memory for plotting
    clawCP.output_format = None   # Don't write solution data to file
    clawCP.solver.dt_initial=1.e99
    status = clawCP.run()
    frameCP_init = clawCP.frames[0]
    densityCP_init = frameCP_init.q[0,:,:]
#   (vxCP_init,vyCP_init) = np.gradient(densityCP_init)
#   vsCP_init = np.sqrt(vxCP_init**2 + vyCP_init**2)
    frameCP_fin = clawCP.frames[-1]
    densityCP_fin = frameCP_fin.q[0,:,:]
#    (vxCP_fin,vyCP_fin) = np.gradient(densityCP_fin)
#    vsCP_fin = np.sqrt(vxCP_fin**2 + vyCP_fin**2)  
    errCP[i-1]=np.linalg.norm((densityCP_init-densityCP_fin).flatten(),ord=1)*(10**2)/(10*i)**2

In [ ]:
ns=range(10,101,10)
plt.plot(ns,errSC,label=('SharpClaw'))
plt.plot(ns,errCP,label=('ClawPack'))
plt.yscale('log')
plt.xscale('log')
plt.title('Errors in the density with 100 time steps')
plt.xlabel('Grid size')
plt.ylabel('L1 norm of error')
plt.legend()

Interestingly, we can observe that the Sharpclaw might be less accurate than standard Clawpack for bigger grid sizes. We suspect that that passing through the boundaries may play a role, so we repeat the error analysis, but for mean velocities $(0,0)$.

In [ ]:
errSC_noBC=[0]*10
for i in range (1,11):
    clawSC = pyclaw.Controller()
    clawSC.tfinal = 10.
    clawSC.num_output_times =100

    clawSC.solver = pyclaw.SharpClawSolver2D(riemann_solver)
    clawSC.solver.all_bcs = pyclaw.BC.periodic

    grid_size = (i*10,i*10)
    domain = pyclaw.Domain( (0.,0.), (10.,10.), grid_size)

    clawSC.solution = pyclaw.Solution(clawSC.solver.num_eqn,domain)
    gam = 1.4
    clawSC.solution.problem_data['gamma']  = gam

# Set initial data
    q = clawSC.solution.q
    xx,yy = domain.grid.p_centers
    eps=5 #vortex strength
    xbar=xx-5.0
    ybar=yy-5.0
    r2=xbar**2+ybar**2
    A=eps/(2*np.pi)*np.exp(0.5*(1.0-r2))
    dT=-(gam-1.)*eps**2/(8*gam*np.pi**2)*np.exp(1-r2)
    S_vor=1
    T=1+dT
    rho  = pow(T / S_vor, 1 / (gam - 1.));
    P=rho*T
    c_v=0
    c_w=0

    q[0,...] =rho
    q[1,...] = rho*(c_v-A*ybar)
    q[2,...] = rho*(c_w+A*xbar)
    q[3,...] = 0.5*rho*((c_v-A*ybar)**2+(c_w+A*xbar)**2) + P/(gam-1.)

    clawSC.keep_copy = True      # Keep solution data in memory for plotting
    clawSC.output_format = None   # Don't write solution data to file
    clawSC.solver.dt_initial=1.e99
    status = clawSC.run()
    frameSC_init = clawSC.frames[0]
    densitySC_init = frameSC_init.q[0,:,:]
#    (vxSC_init,vySC_init) = np.gradient(densitySC_init)
#    vsSC_init = np.sqrt(vxSC_init**2 + vySC_init**2)
    frameSC_fin = clawSC.frames[-1]
    densitySC_fin = frameSC_fin.q[0,:,:]
#    (vxSC_fin,vySC_fin) = np.gradient(densitySC_fin)
#    vsSC_fin = np.sqrt(vxSC_fin**2 + vySC_fin**2)  
    errSC_noBC[i-1]=np.linalg.norm((densitySC_init-densitySC_fin).flatten(),ord=1)*(10**2)/(i*10)**2

errCP_noBC=[0]*10
for i in range (1,11):
    clawCP = pyclaw.Controller()
    clawCP.tfinal = 10.
    clawCP.num_output_times =100

    clawCP.solver = pyclaw.ClawSolver2D(riemann_solver)
    clawCP.solver.all_bcs = pyclaw.BC.periodic

    grid_size = (i*10,i*10)
    domain = pyclaw.Domain( (0.,0.), (10.,10.), grid_size)

    clawCP.solution = pyclaw.Solution(clawCP.solver.num_eqn,domain)
    gam = 1.4
    clawCP.solution.problem_data['gamma']  = gam

# Set initial data
    q = clawCP.solution.q
    xx,yy = domain.grid.p_centers
    eps=5 #vortex strength
    xbar=xx-5.0
    ybar=yy-5.0
    r2=xbar**2+ybar**2
    A=eps/(2*np.pi)*np.exp(0.5*(1.0-r2))
    dT=-(gam-1.)*eps**2/(8*gam*np.pi**2)*np.exp(1-r2)
    S_vor=1
    T=1+dT
    rho  = pow(T / S_vor, 1 / (gam - 1.));
    P=rho*T
    c_v=0
    c_w=0

    q[0,...] =rho
    q[1,...] = rho*(c_v-A*ybar)
    q[2,...] = rho*(c_w+A*xbar)
    q[3,...] = 0.5*rho*((c_v-A*ybar)**2+(c_w+A*xbar)**2) + P/(gam-1.)

    clawCP.keep_copy = True      # Keep solution data in memory for plotting
    clawCP.output_format = None   # Don't write solution data to file
    clawCP.solver.dt_initial=1.e99
    status = clawCP.run()
    frameCP_init = clawCP.frames[0]
    densityCP_init = frameCP_init.q[0,:,:]
#    (vxCP_init,vyCP_init) = np.gradient(densityCP_init)
#    vsCP_init = np.sqrt(vxCP_init**2 + vyCP_init**2)
    frameCP_fin = clawCP.frames[-1]
    densityCP_fin = frameCP_fin.q[0,:,:]
#    (vxCP_fin,vyCP_fin) = np.gradient(densityCP_fin)
#    vsCP_fin = np.sqrt(vxCP_fin**2 + vyCP_fin**2)  
    errCP_noBC[i-1]=np.linalg.norm((densityCP_init-densityCP_fin).flatten(),ord=1)*(10**2)/(10*i)**2



In [ ]:
ns=range(10,101,10)
plt.plot(ns,errSC_noBC,label=('SharpClaw'))
plt.plot(ns,errCP_noBC,label=('ClawPack'))
plt.yscale('log')
plt.xscale('log')
plt.title('Errors in the density with 100 time steps, zero velocity')
plt.xlabel('Grid size')
plt.ylabel('L1 norm of error')
plt.legend()

So with zero mean velocities (i.e. the vortex is the center of reference), we see that Sharpclaw is uniformly more accurate that standard Clawpack as expected. 

In order to be more sure that the problem is indeed with boundaries, we let the vortex travel from (4,4) to (6,6). We compute the error in the same way, but now comparing submatrices (50% of the whole domain) such that the initial and the final vortices will be in the centers respectively.

Since the distance travelled is shorter, we run experiment for larger domain sizes to get more definitive conclusion.

We see that for the short distance, Sharpclaw will perform better and seems like standard Clawpack will serve as threshold (at least for this range). We are not sure if those lines will meet and we do not know the oscillatory nature of the Sharpclaw. It would be interesting to invesitgate further this issue.

In [ ]:
errSC_SmallDist=[0]*60
for i in range (1,61):
    clawSC = pyclaw.Controller()
    clawSC.tfinal = 2.
    clawSC.num_output_times =20

    clawSC.solver = pyclaw.SharpClawSolver2D(riemann_solver)
    clawSC.solver.all_bcs = pyclaw.BC.periodic

    grid_size = (i*10,i*10)
    domain = pyclaw.Domain( (0.,0.), (10.,10.), grid_size)

    clawSC.solution = pyclaw.Solution(clawSC.solver.num_eqn,domain)
    gam = 1.4
    clawSC.solution.problem_data['gamma']  = gam

# Set initial data
    q = clawSC.solution.q
    xx,yy = domain.grid.p_centers
    eps=5 #vortex strength
    xbar=xx-4.0
    ybar=yy-4.0
    r2=xbar**2+ybar**2
    A=eps/(2*np.pi)*np.exp(0.5*(1.0-r2))
    dT=-(gam-1.)*eps**2/(8*gam*np.pi**2)*np.exp(1-r2)
    S_vor=1
    T=1+dT
    rho  = pow(T / S_vor, 1 / (gam - 1.));
    P=rho*T
    c_v=1
    c_w=1

    q[0,...] =rho
    q[1,...] = rho*(c_v-A*ybar)
    q[2,...] = rho*(c_w+A*xbar)
    q[3,...] = 0.5*rho*((c_v-A*ybar)**2+(c_w+A*xbar)**2) + P/(gam-1.)

    clawSC.keep_copy = True      # Keep solution data in memory for plotting
    clawSC.output_format = None   # Don't write solution data to file
    clawSC.solver.dt_initial=1.e99
    status = clawSC.run()
    frameSC_init = clawSC.frames[0]
    densitySC_init = frameSC_init.q[0,:,:]
#   (vxSC_init,vySC_init) = np.gradient(densitySC_init)
#    vsSC_init = np.sqrt(vxSC_init**2 + vySC_init**2)
    frameSC_fin = clawSC.frames[-1]
    densitySC_fin = frameSC_fin.q[0,:,:]
#    (vxSC_fin,vySC_fin) = np.gradient(densitySC_fin)
#   vsSC_fin = np.sqrt(vxSC_fin**2 + vySC_fin**2)  
#    errSC_SmallDist[i-1]=np.linalg.norm((densitySC_init[1*i:(7*i-1),1*i:(7*i-1)]-densitySC_fin[3*i:(9*i-1),3*i:(9*i-1)]).flatten(),ord=1)*(10**2)/(i*10)**2
    errSC_SmallDist[i-1]=np.linalg.norm((densitySC_init[2*i-1:(6*i-1),2*i-1:(6*i-1)]-densitySC_fin[4*i-1:(8*i-1),4*i-1:(8*i-1)]).flatten(),ord=1)*(10**2)/(i*10)**2

errCP_SmallDist=[0]*60
for i in range (1,61):
    clawCP = pyclaw.Controller()
    clawCP.tfinal = 2.
    clawCP.num_output_times =20

    clawCP.solver = pyclaw.ClawSolver2D(riemann_solver)
    clawCP.solver.all_bcs = pyclaw.BC.periodic

    grid_size = (i*10,i*10)
    domain = pyclaw.Domain( (0.,0.), (10.,10.), grid_size)

    clawCP.solution = pyclaw.Solution(clawCP.solver.num_eqn,domain)
    gam = 1.4
    clawCP.solution.problem_data['gamma']  = gam

# Set initial data
    q = clawCP.solution.q
    xx,yy = domain.grid.p_centers
    eps=5 #vortex strength
    xbar=xx-4.0
    ybar=yy-4.0
    r2=xbar**2+ybar**2
    A=eps/(2*np.pi)*np.exp(0.5*(1.0-r2))
    dT=-(gam-1.)*eps**2/(8*gam*np.pi**2)*np.exp(1-r2)
    S_vor=1
    T=1+dT
    rho  = pow(T / S_vor, 1 / (gam - 1.));
    P=rho*T
    c_v=1
    c_w=1

    q[0,...] =rho
    q[1,...] = rho*(c_v-A*ybar)
    q[2,...] = rho*(c_w+A*xbar)
    q[3,...] = 0.5*rho*((c_v-A*ybar)**2+(c_w+A*xbar)**2) + P/(gam-1.)

    clawCP.keep_copy = True      # Keep solution data in memory for plotting
    clawCP.output_format = None   # Don't write solution data to file
    clawCP.solver.dt_initial=1.e99
    status = clawCP.run()
    frameCP_init = clawCP.frames[0]
    densityCP_init = frameCP_init.q[0,:,:]
#    (vxCP_init,vyCP_init) = np.gradient(densityCP_init)
#    vsCP_init = np.sqrt(vxCP_init**2 + vyCP_init**2)
    frameCP_fin = clawCP.frames[-1]
    densityCP_fin = frameCP_fin.q[0,:,:]
#    (vxCP_fin,vyCP_fin) = np.gradient(densityCP_fin)
#    vsCP_fin = np.sqrt(vxCP_fin**2 + vyCP_fin**2)  
#    errCP_SmallDist[i-1]=np.linalg.norm((densityCP_init[1*i:(7*i-1),1*i:(7*i-1)]-densityCP_fin[3*i:(9*i-1),3*i:(9*i-1)]).flatten(),ord=1)*(10**2)/(10*i)**2
    errCP_SmallDist[i-1]=np.linalg.norm((densityCP_init[2*i-1:(6*i-1),2*i-1:(6*i-1)]-densityCP_fin[4*i-1:(8*i-1),4*i-1:(8*i-1)]).flatten(),ord=1)*(10**2)/(10*i)**2


In [ ]:
ns=range(10,601,10)
plt.plot(ns,errSC_SmallDist,label=('SharpClaw'))
plt.plot(ns,errCP_SmallDist,label=('ClawPack'))
plt.yscale('log')
plt.xscale('log')
plt.title('Errors in the density with 20 time steps, (4,4) to (6,6)')
plt.xlabel('Grid size')
plt.ylabel('L1 norm of error')
plt.legend()